# Code de Huffman

Intro:
J'ai choisi de l'appliquer à des chaines de caractéres.

Rq : J'ai choisi d'écrire les algorithmes écrits en langage naturel, car le pseudo code n'est pas au programme de NSI. Dans le programme il est indiqué que python étant très proche du pseudo code on peut s'en passer. 

In [2]:
phrase = "Le covid c'est pas fantastique, les vacances on travaille le projet"

# I. Compression

### Etapes de la compression : ###
[1. obtenir les occurences des caractéres de la chaine de caractére](#occurence)    
[2. construire l'arbre de Huffmann](#contruction_arbre)    
   * [je propose une version en utilisant la structure "tas", c'est ce qu'on trouve classiquement ;-)](#arbre_avec_tas)
   * [une autre sans les tas, avec des listes](#arbre_avec_liste)
   
3. calcul de la table de codage par un parcours d’arbre
4. utilisation du code 


<div id="occurence">
<h1>1. Ocurences des caractéres</h1>
    
Pour former l'arbre de Huffman, il faut déterminer les occurences de chacun des caractéres apparaissant dans le texte.
Ci dessous, une fonction qui renvoie un dictionnaire contenant les occurences de chacun des caractéres présents d'une chaine de caractére (les clés sont les caractéres, les valeurs sont les occurences) puis une fonction pour convertir le dictionnaire en liste, ou en tas, pour la suite

In [3]:
def creationDictionnaireOccurence(u):
    '''Entrée : u : une chaine de caractére e
    Sortie : dictionnaire {caractére : occurence}'''
    occu = {}
    for c in u :
        if c in occu : #si la clé n'existe pas, on la crée et on met la valeur à 1
            occu[c]+=1
        else :
            occu[c]=1 #sinon on ajoute 1 la valeur de la clé
    return occu
    

<div class="alert alert-warning" role="alert">
Correction :   
    
  - Terminaison : boucle for qui se termine après O(n) opérations(avec n  longueur de la chaine de caractéres, car on itére sur chacun des caractéres de la chaine.
     
  - Validité  : invariant de boucle : Après k itérations, le dictionnaire contient les occurences des caractéres présents dans la partie de la chaine du début jusqu'au  kéme caractéres. (Puis par récurence...)   

In [4]:
def conversionDictionnaireOccurenceEnListe(dico):
    '''Entrée : u : une chaine de caractére e
    Sortie : liste contenant des tuples(occurence,('F', caractere)'''
    liste = [(occurence,('F', lettre)) for lettre,occurence in dico.items()]
    return liste

<div class="alert alert-warning" role="alert">
Correction  :  
    
   - Terminaison : la conversion du dictionnaire se termine après O(n) (n longueur de la chaine de carctére, car on itére sur chacun items du tableau, et au pire, il y a autant d'items que de caractére, si la chaine de caractére de départ ne contient que des caractéres différents.)
   
   - Validité  : invariant de boucle : Après k itérations, la liste contient les occurences de k caractéres du dictionnaire  . (Puis par récurence...)   

In [5]:
def conversionDictionnaireOccurenceEnTas(dico):
    tas = [(occurence,('F', lettre)) for lettre,occurence in dico.items() ]#on convertit le dictionnaire en liste
    convertirListeEnTas(tas)# on "tamise" la liste pour en faire un tas (voir plus bas la notion de tas)
    return tas

<div class="alert alert-warning" role="alert">
Correction  :  
    
   - Terminaison : la conversion du dictionnaire en liste se termine après O(n) opérations (n longueur de la chaine de caractére, car on itére sur chacun  des items du dictionnaire, et au pire, il y a autant d' item que de caractére, si la chaine de caractére de départ ne contient que des caractéres différents. Et la fonction `heapify` est en O(n) aussi <a herf="#arbre_avec_tas">démonstration plus bas</a>. Donc la fonction a une complexité en O(n)
   
   - Validité : invariant de boucle : Après k itérations, la liste contient les occurences de k caractéres du dictionnaire  . (Puis par récurence...). Vois validite de la fonction `heapify` <a herf="#arbre_avec_tas"> plus bas</a>
    

<div id ="contruction_arbre">
    <h1> 2. La construction de l'arbre de Huffman</h1>
    

<div class="alert alert-success" role="alert">
  <h4 class="alert-heading">Comment j'ai choisi de représenter l'arbre de Huffman en Python ?</h4>


* L'arbre vide est codé par le tuple vide () .  
* les feuilles seront représentées par le t-uplet (occurence,('F', a)) ( F comme "feuille").   
* Les noeuds ont un fils gauche et un fils droit . Soient G et D leurs représentations :   
les noeuds seront représentés par le t-uplet (somme des occurences,('N', G, D)) ( N comme "noeud").

> Rq : J'ai choisi d'utiliser des t-uplets pour représenter les feuilles et les noeuds plutot que des dictionnaires pour m'affranchir du problème lors de la comparaison par la fonction heapq.heappush.
En effet pour "ranger le tas correctement", la fonction compare (strictement) les clés de priorité, et en cas d'égalité sur la valeur de la clé de priorité, va comparer la valeur suivante de l'élement, mais on ne peut pas comparer des dictionnaires en python ! Par contre on peut comparer des listes...

<div id = "arbre_avec_tas">
<h2 > Construction de l'arbre en utilisant un tas</h2>


Pour fabriquer l'arbre de Huffman associé au dictionnaire contenant les occurences des caractéres, on peut utiliser un tas.
On lit dans wikipédia :
>"En informatique, un tas (ou monceau au Canada2, heap en anglais) est une structure de données de type arbre qui permet de retrouver directement l'élément que l'on veut traiter en priorité.  

C'est ce que l'on veut dans la construction de l'arbre de Huffman !

En effet un arbre de Huffmann est un arbre binaire, et on veut pouvoir traiter les noeuds de poids (occurence) faibles. Ce que permet typiquement un arbre.

>On dit qu'un arbre est ordonné en tas lorsque la propriété suivante est vérifiée :
>
>    Pour tous nœuds A et B de l'arbre tels que B soit un fils de A
>    clé(A) ≥ clé(B)
>
>ou
>
>    Pour tous nœuds A et B de l'arbre tels que B soit un fils de A
>    clé(A) ≤ clé(B)
>
>Un arbre vérifiant cette propriété est aussi appelé « arbre tournoi »4. Cette propriété implique que la plus grande clé (ou la plus petite) soit située à la racine du tas. Ils sont ainsi très utilisés pour implémenter les files à priorités car ils permettent des insertions en temps logarithmique et un accès direct au plus grand élément. L'efficacité des opérations effectuée sur des tas est très importante dans de nombreux algorithmes sur les graphes. Les tas sont en outre utilisés dans l'algorithme de tri par tas.
>
>On peut ainsi représenter un tas à n éléments par un tableau à n cases, de la façon suivante :
>
>Si on note 0 le numéro de la racine du tas, on peut définir récursivement le numéro de tous les sommets dans l'arbre, grâce à la formule : Pour un sommet parent numéroté i, son fils gauche aura pour numéro 2*i+1, et son sommet droit 2*i+2. Le parent d'un sommet j a donc toujours pour numéro la partie entière inférieure de (j-1)/2.   

<img src="heap.png">

<a href ="https://towardsdatascience.com/data-structure-heap-23d4c78a6962">source</a>  

Un tas est une structure de données permettant les opérations suivantes :  
- Mettre dans le tas un objet avec une priorité donnée (une priorité est, par exemple, un entier, ici ce sera l'occurence ou la somme des occurences des fils)
- Retirer de la file l'objet de priorité minimale.


> RQ : Pour le tri dans le tas    
A valeur de clé de priorité (occurence (ou somme des occurences)) égale, l'emploi de p-uplet permet bien l'ordonnancement car, ce sera F avant N (par ordre alphabétique), et pour deux feuilles, ordre alphabétique du caractére, pour deux noeuds ordre alhabétique du fils gauche... ). Le caractére étant unique dans l'arbre, l'ordonnancement sera assuré.  

<div class="alert alert-danger" role="alert">
Voici l'algorithme, pour la construction de l'arbre en utilisant les tas :
1. On met dans le tas les futures feuilles de notre arbre avec leurs occurence comme clé de priorité. Pour cela il suffit de lire le dictionnaire des occurences dont les clés sont les lettres et les valeurs sont les occurences. (Fais en haut dans la fonction conversionDictionnaireOccurenceEnTas)
2. Pour k allant de 0 au nombre de feuilles -1 :  (il y a n feuilles, il y aura n-1 noeuds binaires) 
   * On retire du tas la racine ( et on réarange le tas), soit x cet élément. 
   * On retire du nouveau tas la racine ( et on réarange le tas), soit y cet élément. 
   * On crée le noeud z dont les deux fils sont x et y, et de priorité la somme des priorités des fils .   
   * On met z dans le tas correctement rangé.   
3. Maintenant, le tas contient un unique arbre. On renvoie cet arbre.   

Il faut donc :
 * une fonction pour transformer une liste en tas, c'est ce que fait la fonction `convertirListeEnTas` 
 * une fonction pour retirer la racine du tas (occurence la plus petite) et réaranger le tas, c'est ce que fait la fonction `suppressionDeLaRacine`
 * une fonction pour ajouter un noeud dans le tas et réaranger le tas, c'est ce que fait la fonction `insererElement`
 

### Les fonctions nécessaires pour manipuler les tas binaires ###

Rq : hauteur d'un tas : h (un tas est un arbre binaire presque complet)
Un tas T qui emmagasine n élements a une hauteur h = O(log n)
Preuve:   
–Soit h la hauteur d'un tas stockant les n  clés   
–Puisque il y a 2 ** i clés à la profondeur i= 0, ... , h - 2 et au moins une clé à profondeur h - 1, nous avons $ n ≥ 1 + 2 + 4 +...+ 2^{h-2}  + 1  $ 
–Ainsi, $ n ≥  2^{h-1} $  , i.e., h ≤ log(n) + 1

Fonction heapify
entrée : une liste de n éléments
sortie : une liste qui implémente un tas de n éléments

L'algorithme est le suivant :
1. on prend 

Fonction `suppressionDeLaRacine` (c'est à dire du min)        
  + entrée : une liste qui implémente un tas de n éléments
  + sortie : une liste qui implémente un tas constitué des n-1 élements du tas d'entrée moins sa racine **et** la racine supprimée

**Algorithme**
1. On remplace la racine (c'est à dire l'élément 0 de la liste) par le dernier élement du tas 
2. on fait descendre cet élément dans le tas, en réagençant les différents éléments, pour garder la structure de tas.

Pour l'étape 2, il nous faut une procédure `descendre` (de la racine vers les feuilles)
  + Entrée : une liste qui implémente un tas, sauf 1 élément d'indice i **et** l'indice i
  + Sortie : une liste qui implémente un tas
  
**Algorithme**   
Tant que l'on a pas atteint l'avant dernier étage (indice dans la liste <n//2) ou que l'élément n'est pas plus petit que ces deux enfants   
  * On compare l'élement i à ses deux enfants (2*i et 2*i+1)
  * Si l'élément i est plus grand qu'un de ses enfants (ou les deux) on l'échange avec le plus petit de ses enfants et i prend la valeur de l'indice de l'enfant avec qui on l'a échangé


In [6]:
def descendre (presqueTas,i): #on descend l'élément i vers les feuilles jusqu'à ce que l'élément soit dans une position de parent qui respecte la structure de tas
    n= len(presqueTas)
    
    while i<n//2 : #on descends juste avant les feuilles
        indiceEnf1 = 2*i+1 #les enfants sont en 2*i+1 et 2*i+2 le deuxieme enfant n 'existe pas forcement'
        enfant1 = presqueTas[indiceEnf1]
        if indiceEnf1<(n-1) : #cela signifie qu 'il y a le deuxieme enfant'
            indiceEnf2 = 2*i+2 
            
            enfant2 = presqueTas[indiceEnf2]
            if presqueTas[i]>enfant1 or presqueTas[i]>enfant2:
                if (enfant2>enfant1): #on échange avec le plus petit des enfants
                    presqueTas[i],presqueTas[indiceEnf1] = presqueTas[indiceEnf1],presqueTas[i]
                    i =indiceEnf1
                    print ('ienf1',i)
                else :
                    presqueTas[i],presqueTas[indiceEnf2] = presqueTas[indiceEnf2],presqueTas[i]
                    i =indiceEnf2
                    print ('ienf2',i)
                print(tas)
            else : break  
        else : #il n'y a qu'un enfant on vérifie si il faut faire l'échange ou pas 
            if presqueTas[i]>enfant1:
                    presqueTas[i],presqueTas[indiceEnf1] = presqueTas[indiceEnf1],presqueTas[i]
                    i =indiceEnf1
                    print ('ienf1unique',i)
            else: break
            
def _siftup(heap, pos):
    endpos = len(heap)
    startpos = pos
    newitem = heap[pos]
    # Bubble up the smaller child until hitting a leaf.
    childpos = 2*pos + 1    # leftmost child position
    while childpos < endpos:
        # Set childpos to index of smaller child.
        rightpos = childpos + 1
        if rightpos < endpos and not heap[childpos] < heap[rightpos]:
            childpos = rightpos
        # Move the smaller child up.
        heap[pos] = heap[childpos]
        pos = childpos
        childpos = 2*pos + 1
    # The leaf at pos is empty now.  Put newitem there, and bubble it up
    # to its final resting place (by sifting its parents down).
    heap[pos] = newitem
    _siftdown(heap, startpos, pos)
    
    
def suppressionDeLaRacine(tas):
    racine = tas[0]
    tas[0]=tas[len(tas)-1]
    tas.pop()
    descendre (tas,0)
    return (racine)
#### Pour tester :
if True :
    tas = [3,4,6,21,10,7,8,22,28,13,19,25,20]
    for i in range(len(tas)):
        suppressionDeLaRacine(tas)
        print (tas)

ienf1 1
[4, 20, 6, 21, 10, 7, 8, 22, 28, 13, 19, 25]
ienf2 4
[4, 10, 6, 21, 20, 7, 8, 22, 28, 13, 19, 25]
ienf1 9
[4, 10, 6, 21, 13, 7, 8, 22, 28, 20, 19, 25]
[4, 10, 6, 21, 13, 7, 8, 22, 28, 20, 19, 25]
ienf2 2
[6, 10, 25, 21, 13, 7, 8, 22, 28, 20, 19]
ienf1 5
[6, 10, 7, 21, 13, 25, 8, 22, 28, 20, 19]
[6, 10, 7, 21, 13, 25, 8, 22, 28, 20, 19]
ienf2 2
[7, 10, 19, 21, 13, 25, 8, 22, 28, 20]
ienf2 6
[7, 10, 8, 21, 13, 25, 19, 22, 28, 20]
[7, 10, 8, 21, 13, 25, 19, 22, 28, 20]
ienf2 2
[8, 10, 20, 21, 13, 25, 19, 22, 28]
ienf2 6
[8, 10, 19, 21, 13, 25, 20, 22, 28]
[8, 10, 19, 21, 13, 25, 20, 22, 28]
ienf1 1
[10, 28, 19, 21, 13, 25, 20, 22]
ienf2 4
[10, 13, 19, 21, 28, 25, 20, 22]
[10, 13, 19, 21, 28, 25, 20, 22]
ienf1 1
[13, 22, 19, 21, 28, 25, 20]
ienf1 3
[13, 21, 19, 22, 28, 25, 20]
[13, 21, 19, 22, 28, 25, 20]
ienf2 2
[19, 21, 20, 22, 28, 25]
[19, 21, 20, 22, 28, 25]
ienf2 2
[20, 21, 25, 22, 28]
[20, 21, 25, 22, 28]
ienf1 1
[21, 28, 25, 22]
ienf1unique 3
[21, 22, 25, 28]
ienf1 1
[22, 28

<div class="alert alert-warning" role="alert">
Correction de la fonction descendre:  
    
   - Terminaison : 
     - Descendre se terminera après O(log n) opérations car au pire on fera descendre de la racine jusqu'à une feuille l'élément, c'est à dire de toute la hauteur de l'arbre -1 = h-1 qui est O(log n).  
     - Une autre façon de voir, est de definir le convergent : n//2 - i,i, est ua pire des cas initialisé à 0, et i est multiplié par 2 à chaque itération, donc la boucle se termine, et au pire il y aura log(n) itérations.
   
   - Validité : Il faut vérifier qu'à la fin, la liste a bien une structure de tas.
     - Invariant de boucle : A la fin d'une itération, les élèments de la liste autre que i, 2.i+1 et 2.i+2 ont bien une structure de tas, c'est à dire que le parent est inférieur à ces deux enfants.
     - Par récursivité :
       - La première itération, la liste est rangée en tas (par précondition de l'entrée), sauf peut être i et ses enfants, lors de la première itération, on "range" en structure de tas i et ses enfants, et donne à i la valeur de l'indice de l'enfant avec qui il a été echangé. 
       - A chaque tour de boucle, on range correctement, i et ses enfants pour respecter la structure de tas, puis i prend la valeur de l'indice de l'enfant avec qui on l'a échangé, donc, la structure d'arbre est respectée, sauf pour le nouveau i et ses enfants.
       - On sort de la boucle, soit si i est bien rangé, et comme tous les autres éléments sont bien rangés, la liste a bien une structure de tas. Soit si i>=n//2, c'est à dire si l'élément auquel on s'interesse est une feuille. (i>=n//2 ---> 2*i+1 >= n+1 donc il n'y a pas de fils pour l'élément i, et l'élément i est une feuille)
       
> Rq : la correction de la fonction suppressionDeLaRacine, dépend directement de la correction de la fonction descendre

Fonction `insererElement`         
  + entrée : une liste qui implémente un tas de n éléments **et** un élement à ajouter.
  + sortie : une liste qui implémente un tas constitué des n élements du tas d'entrée et de l'élément à ajouter

**Algorithme**
1. On place l'élément à la fin de la liste 
2. on fait monter cet élément dans le tas (vers la racine), en réagençant les différents éléments, pour garder la structure de tas.

Pour l'étape 2, il nous faut une procédure `monter` (des feuilles vers la racine)
  + Entrée : une liste qui implémente un tas, sauf le dernier élément
  + Sortie : une liste qui implémente un tas
  
**Algorithme**   
Tant que l'on a pas atteint la racine (i>=1)  ou que l'élément n'est pas plus grand que son parent   
  * On compare l'élement i à son parent d'indice ((i-1)//2)
  * Si l'élément i est plus petit que son parent on l'échange avec le plus petit de ses enfants, et i prend  la valeur de l'indice du parent

In [7]:
def rajouterElement (tas, e):
    tas.append(e)
    monter(tas)
    return tas

def monter (tas): #on monte le dernier élément jusqu'à une bonne place, vers la racine
    i = len(tas)-1
    while i>=1:
        indiceParent = (i-1)//2
        if tas[i]<tas[indiceParent] :
            tas[i],tas[indiceParent] = tas[indiceParent],tas[i]
            i = indiceParent
        else :
            break

#### Pour tester :
if False :            
    tas = [3,4,6,21,10,7,8,22,28,13,19,25,20]
    rajouterElement(tas,2)
    print (tas)   

<div class="alert alert-warning" role="alert">
Correction de la fonction monter:  
    
   - Terminaison : 
     - Monter se terminera après O(log n) opérations car au pire on fera monter des feuilles jusqu'à la racine l'élément, c'est à dire de toute la hauteur de l'arbre -1 = h-1 qui est O(log n).  
     - Une autre façon de voir, est de definir le convergent : i-3, i est  initialisé à la n (nombre d'élément), et i est divisé par 2 à chaque itération, donc la boucle se termine, et au pire il y aura log(n) itérations.
   
   - Validité : Il faut vérifier qu'à la fin, la liste a bien une structure de tas.
     - Invariant de boucle : A la fin d'une itération, les élèments de la liste autre que i et son parent d'indice (i-1)//2 ont bien une structure de tas, c'est à dire que le parent est inférieur à ces deux enfants.
     > Rq : Et le deuxième fils !
>    - soit le dernier élément enf1 a un frère enf2, et son frere est superieur à son parent par, car la structure de tas est respécté donc enf2>par, si on effectue l'échange entre enf1 et parent, c'est que enf1<par, ce qui implique enf2>enf1 (or enf1 devient le parent de enf2) la structure de tas est respectée. 
>    - Si le dernier élément est fils unique, la question du deuxième enfant ne se pose pas !
> Cela ne se pose que pour le dernier élèment, car un tas est un arbre binaire presque complet.
     - Par récursivité :
       - La première itération, "range", si nécessaire, en structure de tas le dernier élément et son parent (pour le deuxième enfant voir remarque ci-dessus), et donne à i la valeur de l'indice du parent avec qui il a été echangé. 
       - A chaque itération, on range correctement, i et son parent pour respecter la structure de tas, puis i prend la valeur de l'indice du parent avec qui on l'a échangé, donc, la structure d'arbre est respectée, sauf pour le nouveau i et son parent.
       - On sort de la boucle, soit si i est bien rangé, et comme tous les autres éléments sont bien rangés, la liste a bien une structure de tas. Soit si i=0, c'est à dire si l'élément auquel on s'interesse est une racine. 
       
> Rq : la correction de la fonction rajouterElement, dépend directement de la correction de la fonction monter

Fonction `convertirListeEnTas`         
  + entrée : une liste a n éléments (avec une clé de priorité)
  + sortie : une liste qui implémente un tas constitué des n élements de la liste
  
Nous pourrions choisir d'insérer les n éléments un par un (insérerElément en O(log n ), pour chacun de n éléments), mais cela ferait O(n log(n)) opérations.

Il y a mieux !

L'idée est de réaranger récursivement chaque sous arbre dans le tas en commençant par les feuilles


**Algorithme**
On parcourt les éléments en partant de l'avant dernier niveau (donc le niveau qui ne contient pas de feuille, soit d'indice < à n//2 et on remonte j'usqu' à la racine de droite à gauche (donc n//2, n//2-1, n//2-2....0).
    chaque élèment est descendu vers les feuilles, jusqu'à ce qu'il soit à une osition 


In [8]:
def convertirListeEnTas(x):
    """Transform list into a heap, in-place, in O(len(x)) time."""
    n = len(x)
    # Transform bottom-up.  The largest index there's any point to looking at
    # is the largest with a child index in-range, so must have 2*i + 1 < n,
    # or i < (n-1)/2.  If n is even = 2*j, this is (2*j-1)/2 = j-1/2 so
    # j-1 is the largest, which is n//2 - 1.  If n is odd = 2*j+1, this is
    # (2*j+1-1)/2 = j so j-1 is the largest, and that's again n//2-1.
    for i in reversed(range(n//2)):
        descendre(x, i)
#### Pour tester :
if False :
    tas = [1,2,45,8,9,7,6]
    convertirListeEnTas(tas)
    print (tas)

<div class="alert alert-warning" role="alert">
Correction de la fonction convertirListeEnTas:  
    
   - Terminaison : 
     - Boucle for, qui s'effectue n//2 fois, (et la fonction descendre se termine)
     - Complexité : Soit h la hauteur de l'arbre, on numérote les niveau de 0 pour la racine à h-1 pour les feuilles.   
       pour un élément d'un niveau i, il pourra au max descendre de h-i-1 niveaux jusqu'à une feuille. Et pour un niveau i, il y $ 2^i $ éléments. Donc au max, il y aura :  
       
     $ C = \sum_{i=0}^{h-1} (h-i-1). 2^i $ opérations   
     
     On effectue le changement de variable $ j = h-i-1 $
     
     $ C = 2^{h-1} \sum_{j=1}^{h-1} \frac 1 {2^j} $ opérations  
     
     Or $ \sum_{j=1}^{h-1} \frac 1 {2^j} $ converge (série géométrie de raison inf à 1)  
     
     Donc $ C = O (2^{h-1}) $   
     
     Or h <= log n  
     
     Donc C = O(n)
     
     
   
   - Validité : Il faut vérifier qu'à la fin, la liste a bien une structure de tas.
     - Invariant de boucle : Pendant le parcours des éléments on maintient l'invariant suivant : Tous les sous-arbres des élément à droite de l'élément courant (sur le même niveau) sont des tas. A chaque itération, on applique à l'élément courant, la fonction descendre qui place l'élément tel que le sous arbre sous l'élément courant soit un tas. Donc après avoir traité la racine, comme elle vérifie l'invariant, notre arbre est un tas.
       
> Rq : la correction de cette fonction, dépend directement de la correction de la fonction descendre faite plus haut

In [32]:

def faireArbre(tas, dbg=False):#dbg pour l'affichage
    n = len(tas)
    if dbg : print('Tas initial :', tas)
    for k in range(n - 1): # il y a n feuilles, il y aura n-1 noeuds binaires
        x = suppressionDeLaRacine(tas)#on extrait l'arbre de plus petit poids (occurence) et on réordonne le tas 
        y = suppressionDeLaRacine(tas)#on extrait l'arbre de plus petit poids (occurence) et on réordonne le tas 
        ## les deux t-uplets précédents contiennent les 2 arbres de plus petit point du tas 
        z = ('N', x, y) #on crée le noeud
        rajouterElement(tas, (x[0] + y[0], z))#on l'ajoute au tas, et on réarange le tas
        if dbg: print('Étape %d : %s' % (k, tas))
    return suppressionDeLaRacine(tas) #on extrait l'arbre de huffman. A la fin le tas est composé d'une liste avec pour premiere valeur la somme de toutes les occurences, et en deuxième (indice 1) l'arbre proprement dit
#### Pour tester :
if True :
    phrase = "abracadabra"
    dicOccu=creationDictionnaireOccurence(phrase)
    tas = []
    tas = conversionDictionnaireOccurenceEnTas(dicOccu)
    print(tas)
    print("coucou")
    A = faireArbre(tas)
    print (A)

ienf1 3
[]
ienf1 1
[]
ienf2 4
[]
[(1, ('F', 'c')), (1, ('F', 'd')), (2, ('F', 'r')), (2, ('F', 'b')), (5, ('F', 'a'))]
coucou
ienf1 1
[(1, ('F', 'd')), (5, ('F', 'a')), (2, ('F', 'r')), (2, ('F', 'b'))]
ienf1unique 3
ienf1 1
[(2, ('F', 'b')), (5, ('F', 'a')), (2, ('F', 'r'))]
ienf2 2
[(2, ('F', 'r')), (2, ('N', (1, ('F', 'c')), (1, ('F', 'd')))), (5, ('F', 'a'))]
ienf1unique 1
(11, ('N', (5, ('F', 'a')), (6, ('N', (2, ('N', (1, ('F', 'c')), (1, ('F', 'd')))), (4, ('N', (2, ('F', 'b')), (2, ('F', 'r'))))))))


Meme chose meme avec un appel récursif : 

In [10]:
def faire_arbre_recursif(tas, dbg=False):#dbg pour l'affichage
     
    while len(tas)>=2:
        x = suppressionDeLaRacine(tas)#on extrait l'arbre de plus petit poids (occurence) et on réordonne le tas 
        y = suppressionDeLaRacine(tas)#on extrait l'arbre de plus petit poids (occurence) et on réordonne le tas 
        ## les deux t-uplets précédents contiennent les 2 arbres de plus petit point du tas 
        z = ('N', x, y) #on crée le noeud
        rajouterElement(tas, (x[0] + y[0], z))#on l'ajoute au tas, et on réarange le tas
        faire_arbre(tas, dbg=False)
    return tas[0][1]
#### Pour tester :
if True :
    phrase = "abracadabra"
    dicOccu=creationDictionnaireOccurence(phrase)
    tas = []
    tas = conversionDictionnaireOccurenceEnTas(dicOccu)
    print(tas)
    print("coucou")
    A = faire_arbre(tas)
    print (A)

ienf1 3
[]
ienf1 1
[]
ienf2 4
[]
[(1, ('F', 'c')), (1, ('F', 'd')), (2, ('F', 'r')), (2, ('F', 'b')), (5, ('F', 'a'))]
coucou
ienf1 1
[(1, ('F', 'd')), (5, ('F', 'a')), (2, ('F', 'r')), (2, ('F', 'b'))]
ienf1unique 3
ienf1 1
[(2, ('F', 'b')), (5, ('F', 'a')), (2, ('F', 'r'))]
ienf2 2
[(2, ('F', 'r')), (2, ('N', (1, ('F', 'c')), (1, ('F', 'd')))), (5, ('F', 'a'))]
ienf1unique 1
(11, ('N', (5, ('F', 'a')), (6, ('N', (2, ('N', (1, ('F', 'c')), (1, ('F', 'd')))), (4, ('N', (2, ('F', 'b')), (2, ('F', 'r'))))))))


### Construction de l'arbre en utilisant des listes et sans les tas ###

fonction recherche du min   
fonction supprimer des élements d'une liste (on utilisera pop)   
fonction associer deux noeuds   
fonction ajouter un éleemnts dans la liste (append)  

tant que la liste contient plus de 1 élément :
   on recherche les deux plus petits éléments de la liste
   on les supprime de la liste
   on les associe les fusionne
   on les place à la fin de la liste
   
par recursivité :
si le nombre d'élement de la liste est = à 1, on retourne la liste
sinon on appelle la fonction avec la liste dans laquelle on a supprimé les deux plus petits éléments et rajouté, la fusion des deux éléments supprimés

In [11]:
def mini (liste):
    mini = liste[0][0]
    indiceDuMin =0
    for i in range (1,len(liste)):
        if mini>liste[i][0]:
            mini = liste[i][0]
            indiceDuMin = i
            print (indiceDuMin)
    return liste[indiceDuMin],indiceDuMin

def supprimerElemeParIndice (liste,i) :
    liste.pop(i)
    
def fusion (e1,e2):
    e3occu = e1[0]+e2[0]
    return(e3occu ,('N', e1,e2))

def ajouterElement (liste,e) :
    liste.append(e)

In [12]:
from copy import deepcopy
def faireArbreAvecListe (listeOccu):
    liste = deepcopy(listeOccu)
    while (len(liste)>1) :
        elem1,indiceElem1 = mini (liste)
        print('elem1 ',elem1,indiceElem1)
        supprimerElemeParIndice (liste,indiceElem1)

        print (listeOccu)

        elem2,indiceElem2 = mini (liste)
        print('elem2 ',elem2,indiceElem2)
        supprimerElemeParIndice (liste,indiceElem2)
        print (liste)

        elem3 = fusion(elem1,elem2)


        print('elem3 ' ,elem3)
        ajouterElement (liste,elem3)
    return liste[0]

phrase ='abracadabra'
listeOccu=conversionDictionnaireOccurenceEnListe(creationDictionnaireOccurence(phrase))
arbre = faireArbreAvecListe(listeOccu)
print (arbre)
print (listeOccu)

1
3
elem1  (1, ('F', 'c')) 3
[(5, ('F', 'a')), (2, ('F', 'b')), (2, ('F', 'r')), (1, ('F', 'c')), (1, ('F', 'd'))]
1
3
elem2  (1, ('F', 'd')) 3
[(5, ('F', 'a')), (2, ('F', 'b')), (2, ('F', 'r'))]
elem3  (2, ('N', (1, ('F', 'c')), (1, ('F', 'd'))))
1
elem1  (2, ('F', 'b')) 1
[(5, ('F', 'a')), (2, ('F', 'b')), (2, ('F', 'r')), (1, ('F', 'c')), (1, ('F', 'd'))]
1
elem2  (2, ('F', 'r')) 1
[(5, ('F', 'a')), (2, ('N', (1, ('F', 'c')), (1, ('F', 'd'))))]
elem3  (4, ('N', (2, ('F', 'b')), (2, ('F', 'r'))))
1
elem1  (2, ('N', (1, ('F', 'c')), (1, ('F', 'd')))) 1
[(5, ('F', 'a')), (2, ('F', 'b')), (2, ('F', 'r')), (1, ('F', 'c')), (1, ('F', 'd'))]
1
elem2  (4, ('N', (2, ('F', 'b')), (2, ('F', 'r')))) 1
[(5, ('F', 'a'))]
elem3  (6, ('N', (2, ('N', (1, ('F', 'c')), (1, ('F', 'd')))), (4, ('N', (2, ('F', 'b')), (2, ('F', 'r'))))))
elem1  (5, ('F', 'a')) 0
[(5, ('F', 'a')), (2, ('F', 'b')), (2, ('F', 'r')), (1, ('F', 'c')), (1, ('F', 'd'))]
elem2  (6, ('N', (2, ('N', (1, ('F', 'c')), (1, ('F', 'd'))

### Complexité preuve et autre...
1. Construire le tas initial:(O(n))    
2. A chaque iteration ! extraire 2 fois l’arbre de poids min et reorganiser le tas:O(log n)!fabriquer un nouvel arbre a partir des 2 precedents :O(1)!   rajouter ce nouvel arbre au tas:O(logn)   
3. n−1 iterations  

La construction de l’arbre de Huffman est donc en O(nlogn) 

**Construction du dictionnaire**

Pour construire le dictionnaire qui à chaque caractére associe le code binaire, il faut parcourir l'arbre 

In [34]:


def parcours(arbre,chemin,code) :
    '''Parcourt un arbre binaire, et enregistre le chemin suivi '''
    
    # sur une feuille, on peut associer ce qui est dans chemin à la lettre
    if arbre[1][0] =='F' :
        print('feuille')
        code[arbre[1][1]] = chemin
        return
    
    # sinon, on parcourt l'arbre en mémorisant le chemin pris dans la variable prefixe
    
    parcours(arbre[1][1],chemin+'0',code) #fils de gauche
    
    parcours(arbre[1][2],chemin+'1',code)#fils de droite

def code_huffman(arbre) :
    # on remplit le dictionnaire du code d'Huffman en parcourant l'arbre
    code = {}
    if len(arbre[1])>2 :
        parcours(arbre,'',code)
        
    else : #Si le texte ne comporte qu'un seul caractére
        code[arbre[1][1]] = '0'
    return code
phrase ='abracadabra'
listeOccu=conversionDictionnaireOccurenceEnTas(creationDictionnaireOccurence(phrase))
arbre = faireArbre(listeOccu)
print(arbre)
dico={}
dico = code_huffman(arbre)
print('Dico =\n',dico,'\n')

ienf1 3
[]
ienf1 1
[]
ienf2 4
[]
ienf1 1
[]
ienf1unique 3
ienf1 1
[]
ienf2 2
[]
ienf1unique 1
(11, ('N', (5, ('F', 'a')), (6, ('N', (2, ('N', (1, ('F', 'c')), (1, ('F', 'd')))), (4, ('N', (2, ('F', 'b')), (2, ('F', 'r'))))))))
feuille
feuille
feuille
feuille
feuille
Dico =
 {'a': '0', 'c': '100', 'd': '101', 'b': '110', 'r': '111'} 



### Complexité preuve et autre...
Parcours d'un arbre par recursion

## Encodage d'un texte à partir du dictionnaire

A partir d'un fichier 

In [ ]:
def encodageFichier(dico,fichier) :
    '''il faut que le dictionnaire '''
    f_in = open(fichier,'r',encoding='utf-8')
    texte = f_in.read()
    f_in.close()
    
    caracteresPresentsDico = [c for c in dico.keys()]
    binaire = ''
    for car in texte :
        if car in caracteresPresentsDico :
            binaire = binaire + dico[car]
        else :
            binaire = binaire + dico[' ']
    return binaire

A  partir d'un texte contenu dans une variable :

In [37]:
def encodage(dico,texte) :

    caracteresPresentsDico = [c for c in dico.keys()]
    binaire = ''    
    for car in texte :
        if car in caracteresPresentsDico :
            binaire = binaire + dico[car]
        else :
            binaire = binaire + dico[' ']
    return binaire
phrase ='abracadabra'
listeOccu=conversionDictionnaireOccurenceEnTas(creationDictionnaireOccurence(phrase))
arbre = faireArbre(listeOccu)
print(arbre)
dico={}
dico = code_huffman(arbre)
print('Dico =\n',dico,'\n')
binaire = encodage(dico,phrase)
print(binaire)

ienf1 3
[]
ienf1 1
[]
ienf2 4
[]
ienf1 1
[]
ienf1unique 3
ienf1 1
[]
ienf2 2
[]
ienf1unique 1
(11, ('N', (5, ('F', 'a')), (6, ('N', (2, ('N', (1, ('F', 'c')), (1, ('F', 'd')))), (4, ('N', (2, ('F', 'b')), (2, ('F', 'r'))))))))
feuille
feuille
feuille
feuille
feuille
Dico =
 {'a': '0', 'c': '100', 'd': '101', 'b': '110', 'r': '111'} 

01101110100010101101110


## Décodage d'un fichier compressé

Il faut fournir l'arbre de Huffman, car il n'est pas unique (de manière générale, ici l'implémentation faite fourni le m^me arbre à chaque fois, mais de manière générale il exciste plusieurs arbres de Huffman pour une même entrée.

Plusieurs algorythmes suivant qu'on connaisse l'arbre ou le dictionnaire, a discuter....

In [ ]:
def decodageFichier(arbre,fichierCompresse) :

    # recopie du contenu du fichier
    f_in = open(fichierCompresse,'r',encoding='utf-8')
    compresse = f_in.read()
    f_in.close()

    huffman = arbre
    texte = ''

    # parcours de l'arbre d'Huffman en fonction du bit lu 
    for bit in compresse :
        
        if bit == '0' :
            arbre = arbre[1] #fils de gauche
        else : 
            arbre = arbre[2] #fils de droite

        # sur une feuille, on décode une lettre et on repart à la racine
        if arbre[0] =='F' :
            texte = texte + arbre[1]
            arbre = huffman
    return texte

In [45]:
def decodage(arbre,binaire) :

    huffman = arbre
    texte = ''

    # parcours de l'arbre d'Huffman en fonction du bit lu 
    for bit in binaire :
        print (bit)
        if bit == '0' :
            arbre = arbre[1][1] #fils de gauche
        else : 
            arbre = arbre[1][2] #fils de droite

        # sur une feuille, on décode une lettre et on repart à la racine
        if arbre[1][0] =='F' :
            texte = texte + arbre[1][1]
            arbre = huffman
    return texte
phrase ='abracadabra'
listeOccu=conversionDictionnaireOccurenceEnTas(creationDictionnaireOccurence(phrase))
arbre = faireArbre(listeOccu)
print(arbre)
dico={}
dico = code_huffman(arbre)
print('Dico =\n',dico,'\n')
binaire = encodage(dico,phrase)
print(binaire)
print(decodage (arbre,binaire))

ienf1 3
[]
ienf1 1
[]
ienf2 4
[]
ienf1 1
[]
ienf1unique 3
ienf1 1
[]
ienf2 2
[]
ienf1unique 1
(11, ('N', (5, ('F', 'a')), (6, ('N', (2, ('N', (1, ('F', 'c')), (1, ('F', 'd')))), (4, ('N', (2, ('F', 'b')), (2, ('F', 'r'))))))))
feuille
feuille
feuille
feuille
feuille
Dico =
 {'a': '0', 'c': '100', 'd': '101', 'b': '110', 'r': '111'} 

01101110100010101101110
0
1
1
0
1
1
1
0
1
0
0
0
1
0
1
0
1
1
0
1
1
1
0
abracadabra
